In [1]:
# Lab 3 — SBOM & Provenance Verification (All-in-one cell)

import datetime, json
from datetime import datetime as dt

print("✅ Lab 3 environment ready — SBOM & provenance verification.\n")

REPORT = "lab3_sbom_provenance_report.md"

def write_header():
    with open(REPORT, "w", encoding="utf-8") as f:
        f.write("# Lab 3 — SBOM & Provenance Verification\n")
        f.write(f"_Generated: {datetime.datetime.now().isoformat()}_\n\n")

def append_report(title: str, body: str):
    with open(REPORT, "a", encoding="utf-8") as f:
        f.write(f"## {title}\n{body}\n\n")

write_header()

SBOM = [
    {
        "name": "core-llm-model",
        "version": "1.2.0",
        "type": "model",
        "registry": "registry.trusted-ai.com",
        "source_url": "https://registry.trusted-ai.com/models/core-llm-model",
        "license": "Proprietary",
        "hash": "abc123",
        "signed": True,
        "last_scan_date": "2025-01-10"
    },
    {
        "name": "embedding-model-x",
        "version": "0.9.1",
        "type": "model",
        "registry": "huggingface.co",
        "source_url": "https://huggingface.co/vendor/embedding-model-x",
        "license": "Apache-2.0",
        "hash": "def456",
        "signed": False,
        "last_scan_date": "2024-08-01"
    },
    {
        "name": "transformers",
        "version": "4.38.0",
        "type": "library",
        "registry": "pypi.org",
        "source_url": "https://pypi.org/project/transformers/4.38.0",
        "license": "Apache-2.0",
        "hash": "ghi789",
        "signed": False,
        "last_scan_date": "2024-06-15"
    },
    {
        "name": "fastapi",
        "version": "0.110.0",
        "type": "library",
        "registry": "pypi.org",
        "source_url": "https://pypi.org/project/fastapi/0.110.0",
        "license": "MIT",
        "hash": "jkl012",
        "signed": False,
        "last_scan_date": "2025-01-05"
    },
    {
        "name": "custom-llm-plugin",
        "version": "0.3.0",
        "type": "plugin",
        "registry": "unknown-registry.dev",
        "source_url": "https://unknown-registry.dev/u/random/custom-llm-plugin",
        "license": "Unknown",
        "hash": "zzz999",
        "signed": False,
        "last_scan_date": "2023-12-01"
    }
]

print("📦 Loaded mock SBOM for LLM application with", len(SBOM), "components.\n")

TRUSTED_REGISTRIES = {
    "registry.trusted-ai.com": "internal_model_registry",
    "huggingface.co": "public_model_registry",
    "pypi.org": "python_package_registry",
}

MIN_SCAN_DATE = "2024-01-01"

VULN_DB = {
    ("embedding-model-x", "0.9.1"): "Known prompt injection weakness in embedding pipeline.",
    ("transformers", "4.38.0"): "Outdated version with unresolved security advisory.",
}

def is_date_older_than(date_str: str, cutoff: str) -> bool:
    try:
        d = dt.fromisoformat(date_str)
        c = dt.fromisoformat(cutoff)
        return d < c
    except Exception:
        return True

def validate_component(component: dict):
    issues = []
    reg = component.get("registry")
    if reg not in TRUSTED_REGISTRIES:
        issues.append({"type": "untrusted_registry",
                       "message": f"Registry '{reg}' is not in the trusted list."})
    key = (component.get("name"), component.get("version"))
    if key in VULN_DB:
        issues.append({"type": "known_vulnerability",
                       "message": VULN_DB[key]})
    scan_date = component.get("last_scan_date")
    if not scan_date or is_date_older_than(scan_date, MIN_SCAN_DATE):
        issues.append({"type": "stale_scan",
                       "message": f"Last scan date {scan_date} is older than {MIN_SCAN_DATE} or invalid."})
    if not component.get("signed", False) and component.get("type") in ("model", "plugin"):
        issues.append({"type": "unsigned_artifact",
                       "message": "Model / plugin is not signed; provenance is weaker."})
    if not component.get("license") or component["license"].lower() in ("unknown","n/a"):
        issues.append({"type": "unknown_license",
                       "message": "License is unknown or not clearly specified."})
    return issues

def calculate_risk_score(issues: list) -> int:
    score = 0
    for i in issues:
        t = i["type"]
        if t in ("known_vulnerability","untrusted_registry"):
            score += 40
        elif t in ("stale_scan","unsigned_artifact"):
            score += 25
        elif t == "unknown_license":
            score += 10
    return min(score, 100)

results = []
for comp in SBOM:
    comp_issues = validate_component(comp)
    risk = calculate_risk_score(comp_issues)
    entry = {
        "name": comp["name"],
        "version": comp["version"],
        "registry": comp["registry"],
        "issues": comp_issues,
        "risk_score": risk
    }
    results.append(entry)

print("=== SBOM Verification Results ===")
print(json.dumps(results, indent=2), "\n")

append_report(
    "Component Verification Results",
    f"Results:\n```json\n{json.dumps(results, indent=2)}\n```\n"
)

overall_risk = max(r["risk_score"] for r in results) if results else 0
high_risk_components = [r for r in results if r["risk_score"] >= 70]
medium_risk_components = [r for r in results if 40 <= r["risk_score"] < 70]

summary_text = f"""
Overall maximum component risk score: **{overall_risk}** (0–100 scale).

High-risk components (>= 70):
- {", ".join([c['name'] for c in high_risk_components]) or "None"}

Medium-risk components (40–69):
- {", ".join([c['name'] for c in medium_risk_components]) or "None"}
"""

print(summary_text)

append_report("Risk Summary", summary_text)

reflection = """
Key takeaways:
- Not all components came from trusted registries.
- At least one model/library had a known vulnerability in the mock database.
- Some artifacts were unsigned or had stale scan dates.
- SBOMs plus provenance rules help you answer: 'Do we trust every piece of this LLM stack?'

Reflection questions:
- How would you enforce trusted registries in CI/CD?
- What should happen when a high-risk component is detected?
- How often should SBOMs and scan dates be refreshed in production systems?
"""

append_report("Reflection & Next Steps", reflection)

print("🎯 Lab 3 complete — open Files pane → download lab3_sbom_provenance_report.md.")


✅ Lab 3 environment ready — SBOM & provenance verification.

📦 Loaded mock SBOM for LLM application with 5 components.

=== SBOM Verification Results ===
[
  {
    "name": "core-llm-model",
    "version": "1.2.0",
    "registry": "registry.trusted-ai.com",
    "issues": [],
    "risk_score": 0
  },
  {
    "name": "embedding-model-x",
    "version": "0.9.1",
    "registry": "huggingface.co",
    "issues": [
      {
        "type": "known_vulnerability",
        "message": "Known prompt injection weakness in embedding pipeline."
      },
      {
        "type": "unsigned_artifact",
        "message": "Model / plugin is not signed; provenance is weaker."
      }
    ],
    "risk_score": 65
  },
  {
    "name": "transformers",
    "version": "4.38.0",
    "registry": "pypi.org",
    "issues": [
      {
        "type": "known_vulnerability",
        "message": "Outdated version with unresolved security advisory."
      }
    ],
    "risk_score": 40
  },
  {
    "name": "fastapi",
    "vers